<a href="https://colab.research.google.com/github/auzuha/GANs/blob/main/MNISTGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import numpy as np

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
  torch.cuda.empty_cache()
transform = transforms.Compose([transforms.ToTensor() , transforms.Normalize((0.5,),(0.5,))])
train_data = datasets.MNIST(root="imdata/",download=True,train=True,transform=transform)



Extracting imdata/MNIST/raw/train-images-idx3-ubyte.gz to imdata/MNIST/raw




Extracting imdata/MNIST/raw/train-labels-idx1-ubyte.gz to imdata/MNIST/raw




Extracting imdata/MNIST/raw/t10k-images-idx3-ubyte.gz to imdata/MNIST/raw




Extracting imdata/MNIST/raw/t10k-labels-idx1-ubyte.gz to imdata/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
train_loader = DataLoader(train_data,128,shuffle=False)
#latent vector depth
latent_size = 100
lr=0.0002
#fixed noise vector for visualizing training progress
_latent_vector = np.random.uniform(-1,1,size=(128,100,1,1))
_latent_vector = torch.from_numpy(_latent_vector).to(device)

In [ ]:


#DCGAN Standard Architecture with a few adjustments to match data shape
disc = nn.Sequential(
    nn.Conv2d(1, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Conv2d(128, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1)),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(negative_slope=0.2),
    nn.Conv2d(512 , 1 , 3 ),
    nn.Sigmoid()
)

gen = nn.Sequential(
    nn.ConvTranspose2d(100, 512, kernel_size=(3,3), stride=(1, 1)),
    nn.BatchNorm2d(512),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(512, 256, kernel_size=(3,3), stride=(2, 2), padding=(1, 1)),
    nn.BatchNorm2d(256),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(256, 128, kernel_size=(2,2), stride=(2, 2), padding=(1, 1)),
    nn.BatchNorm2d(128),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(128, 64, kernel_size=(2,2), stride=(2, 2), padding=(1, 1)),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace=True),
    nn.ConvTranspose2d(64, 1, kernel_size=(4,4), stride=(2, 2), padding=(1, 1)),
    nn.Tanh()   
)


gen.to(device)
disc.to(device)

True

True

In [ ]:
#loss function Binary Cross Entropy
criterion = nn.BCELoss()


#optimizers for discriminator and generator
disc_optim = torch.optim.Adam(disc.parameters() , lr)
gen_optim = torch.optim.Adam(gen.parameters() , lr)

In [ ]:
#loss functions to provide labels
def real_loss(preds,smooth=False):
    labels = torch.ones((preds.shape)).to(device)
    if smooth:
      labels = labels*0.9
    loss = criterion(preds,labels)
    return loss
def fake_loss(preds):
    labels = torch.zeros((preds.shape)).to(device)
    loss = criterion(preds,labels)
    return loss




In [ ]:
batch = 0
for epoch in range(1000):
    for xb,yb in train_loader:
        xb=xb.to(device)
        yb=yb.to(device)

        #xb = xb*2 - 1
        ##Discriminator
        disc_optim.zero_grad()
        #real images
        xb=xb.float()
        yb=yb.float()
        preds = disc(xb)
        real_l = real_loss(preds,True)
        #fake images
        ###vcetor
        latent_vector = np.random.uniform(-1,1,size=(len(yb),100,1,1))
        latent_vector = torch.from_numpy(latent_vector).to(device)
        #generate fake images using latent vector (noise) 
        fakes = gen(latent_vector.float())
        fake_preds = disc(fakes.float())
        fake_l = fake_loss(fake_preds)
        total_loss = real_l + fake_l
        total_loss.backward()
        disc_optim.step()

        ##GENERATOR
        gen_optim.zero_grad()
        #latent_vector = np.random.uniform(-1,1,size=(len(yb),100))
        #latent_vector = torch.from_numpy(latent_vector)
        ##latent_vector = torch.randint(0,2,(len(yb),latent_size,1,1)).cuda()
        fakes = gen(latent_vector.float()) #generate fake images
        fake_preds = disc(fakes.float())
        gen_loss = real_loss(fake_preds)
        gen_loss.backward()
        gen_optim.step()        
        fakes = gen(_latent_vector.float())
        copyfakes = fakes[:64]
        torchvision.utils.save_image(copyfakes,f"drive/MyDrive/imgs3/epoch_{epoch+1}_{batch}.jpg")
        batch += 1
         
    print(f"Epoch{epoch+1}/{num_epochs} , disc_loss:{total_loss} ,gen_loss:{gen_loss}")
    
    '''
    plt.imshow(fakes[0].reshape(28,28).detach().cpu(),cmap="gray")
    #plt.imshow(fakes[0].permute(1,2,0).detach().cpu(),cmap="gray")
    plt.title(f"epoch : {epoch+1}")
    plt.show()'''


tensor(0.3835, device='cuda:0', grad_fn=<AddBackward0>) tensor(5.2192, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 469
tensor(0.3650, device='cuda:0', grad_fn=<AddBackward0>) tensor(5.4871, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 938
tensor(0.3627, device='cuda:0', grad_fn=<AddBackward0>) tensor(7.7910, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 1407
tensor(0.5586, device='cuda:0', grad_fn=<AddBackward0>) tensor(3.5929, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 1876
tensor(0.3896, device='cuda:0', grad_fn=<AddBackward0>) tensor(8.3056, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 2345
tensor(0.4062, device='cuda:0', grad_fn=<AddBackward0>) tensor(8.8338, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 2814
tensor(0.4390, device='cuda:0', grad_fn=<AddBackward0>) tensor(4.5569, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward>) 3283
tensor(0.4084, device='cuda:0', grad_fn=<AddBackward0>) tensor(6.9672, device='cuda:0